# Testing

In [7]:
import tweepy
import os
import openpyxl
from dotenv import load_dotenv
import os

load_dotenv()

True

In [8]:
consumer_key = os.getenv("CONSUMER_API_KEY")
consumer_secret = os.getenv("CONSUMER_API_SECRET")
access_token = os.getenv("ACCESS_TOKEN")
access_token_secret = os.getenv("ACCESS_TOKEN_SECRET")
bearer_token = os.getenv("BEARER_TOKEN")

In [9]:
client = tweepy.Client(
    bearer_token=bearer_token,
    consumer_key=consumer_key,
    consumer_secret=consumer_secret,
    access_token=access_token,
    access_token_secret=access_token_secret
)

# Get My User ID

In [4]:
def get_my_user_id():
    """Retrieve the authenticated user's ID."""
    try:
        user = client.get_me()
        print(user)
        return user.data.id if user.data else None
    except Exception as e:
        print(f"An error occurred while retrieving user ID: {e}")
        return None
    
get_my_user_id()

Response(data=<User id=1917558912703684608 name=Shift username=Shift1646020>, includes={}, errors=[], meta={})


1917558912703684608

# Get Replies on POST

In [8]:
import tweepy

# ----------------> Updated get_replies_to_tweets <----------------
def get_replies_to_tweets(tweets_info):
    all_replies = []

    for tweet in tweets_info:
        tweet_id = tweet['tweet_id']
        post_text = tweet['text']
        query = f'conversation_id:{tweet_id} -is:retweet'

        try:
            for response in tweepy.Paginator(
                client.search_recent_tweets,
                query=query,
                tweet_fields=['author_id', 'created_at', 'in_reply_to_user_id'],
                expansions='author_id',
                user_fields=['username'],
                max_results=100
            ):
                if response.data:
                    users = {u['id']: u for u in response.includes['users']}
                    for reply in response.data:
                        author = users.get(reply.author_id)
                        if author:
                            all_replies.append({
                                'conversation_id': tweet_id,
                                'original_post_text': post_text,
                                'reply_tweet_id': reply.id,
                                'username': author.username,
                                'created_at': reply.created_at,
                                'text': reply.text
                            })
        except Exception as e:
            print(f"An error occurred while fetching replies for tweet {tweet_id}: {e}")

    return all_replies



replies = get_replies_to_tweets(top3_tweets)
# for reply in replies:
#     print(f"Reply ID: {reply['tweet_id']}, @{reply['username']} at {reply['created_at']}: {reply['text']}")
type(replies)

list

In [9]:
replies

[{'conversation_id': 1918317966707528131,
  'original_post_text': "🚀 🚀 Crypto enthusiasts are waiting to see if Bitcoin can surpass its record high of $109,135 set earlier this year. 📈 [Nasdaq's filing aims to add popular meme coin to growing crypto ETF lineup](https://t.co/ExAHmg8782) #Bitcoin #ETFs",
  'reply_tweet_id': 1918319826722033724,
  'username': 'Shift1646020',
  'created_at': datetime.datetime(2025, 5, 2, 15, 1, 2, tzinfo=datetime.timezone.utc),
  'text': '@JAlphasquad price action is above 109k narratives are irrelevant trade what you see'},
 {'conversation_id': 1918317966707528131,
  'original_post_text': "🚀 🚀 Crypto enthusiasts are waiting to see if Bitcoin can surpass its record high of $109,135 set earlier this year. 📈 [Nasdaq's filing aims to add popular meme coin to growing crypto ETF lineup](https://t.co/ExAHmg8782) #Bitcoin #ETFs",
  'reply_tweet_id': 1918318465901052249,
  'username': 'JAlphasquad',
  'created_at': datetime.datetime(2025, 5, 2, 14, 55, 37, tzinfo=

# EXCEL Usernames extractions

In [7]:
import pandas as pd
from urllib.parse import urlparse, unquote

def extract_usernames_from_excel(file_path):
    df = pd.read_excel(file_path)
    
    if 'Profile URL' not in df.columns:
        raise ValueError("The Excel file must contain a 'Profile URL' column.")
    
    usernames = []
    for url in df['Profile URL']:
        if isinstance(url, str):
            parsed_url = urlparse(url)
            path = parsed_url.path
            
            if path.startswith('/search') or path == '/':
                continue
            
            segments = path.strip('/').split('/')
            if segments:
                username = unquote(segments[0])
                usernames.append(username)
    
    return usernames

excel_file = 'data/MIND.xlsx'
usernames = extract_usernames_from_excel(excel_file)
usernames

['flybridge',
 'Vikcyter_2',
 'BRICSinfo',
 'rocketonxyz',
 'Permissionless',
 'TheMoonShow',
 'seedphrase',
 'tether_to',
 'Polymarket',
 'Punk583',
 'R3ACHNTWRK',
 'RobertKennedyJr',
 'oakcurrency',
 'nftfactoryparis',
 '0xPolygonFdn',
 'Solana',
 'BSCNheadlines',
 'jokerace_io',
 'X',
 'JackPosobiec',
 'dexter_cap',
 'divine_economy',
 '1PercentBetterX',
 '0xUnicorn_',
 'deekaymotion',
 'cryptonews',
 'surgence_io',
 'legendarygainz_',
 'Stripe',
 'BackedVC',
 'OthersideMeta',
 'iamneubert',
 'wallet',
 'TrustWallet',
 'BorednHungry',
 '11x_official',
 'xAI',
 'ApeCoin',
 'ton_blockchain',
 'streamhq_xyz',
 'banterboycrypto',
 'NFCSummit',
 'Lacoste',
 'DelGroyp',
 'PayPal',
 'jfx',
 'SOCKETProtocol',
 'bear_brand96',
 'get_wildfire',
 'layer3xyz',
 'cartainc',
 'delegatedotxyz',
 'paulg',
 'IdMintThat',
 'jacqmelinek',
 'HumanEvents',
 'Porsche',
 'IamNomad',
 'HexicanHeartel',
 'JimDaily',
 'DCGco',
 'Layer3FDN',
 'p_ferragu',
 'HendrixxWeb3',
 'BochainCapital',
 'BaycCouncil',
 '

# Tweets I am mentioned

In [15]:
def extract_mentions():
    try:
        username = "Shift1646020"

        # Get user ID
        user = client.get_user(username=username)
        if not user.data:
            return []
        user_id = user.data.id

        # Fetch mentions
        mentions = client.get_users_mentions(
            id=user_id,
            max_results=100,
            expansions=['author_id', 'referenced_tweets.id.author_id'],
            tweet_fields=['created_at', 'referenced_tweets'],
            user_fields=['username']
        )

        mention_details = []
        if not mentions.data:
            return mention_details

        # Map author IDs to usernames
        author_ids = {user.id: user.username for user in mentions.includes.get('users', [])}

        # Process each mention
        for tweet in mentions.data:

            author_username = author_ids.get(tweet.author_id, 'Unknown')
            parent_author_id = None
            parent_post_text = None

            if tweet.referenced_tweets:
                for ref_tweet in tweet.referenced_tweets:
                    if ref_tweet.type in ['replied_to', 'quoted']:
                        ref_tweet_id = ref_tweet.id
                        ref_tweet_data = client.get_tweet(
                            id=ref_tweet_id,
                            tweet_fields=['author_id', 'text']
                        )
                        if ref_tweet_data.data:
                            parent_author_id = ref_tweet_data.data.author_id
                            parent_post_text = ref_tweet_data.data.text
                        else:
                            continue
            else:
                parent_author_id = tweet.author_id
                parent_post_text = tweet.text

            if parent_author_id and parent_author_id != user_id:
                mention_details.append({
                    'username': author_username,
                    'tweet_id': tweet.id,
                    'mention_text': tweet.text,
                    'created_at': tweet.created_at,
                    'parent_post_text': parent_post_text
                })
            else:
                print(f"Skipping mention: tweet_id={tweet.id}, parent_author_id={parent_author_id} is user or invalid")

        return mention_details

    except tweepy.TweepyException as e:
        return []
    except Exception as e:
        return []


In [16]:
extract_mentions()

[]

# Comparison

In [10]:
def filter_replies_by_usernames(replies, target_usernames):
    filtered_replies = []

    for reply in replies:
        if reply['username'] in target_usernames:
            filtered_replies.append({
                'tweet_id': reply['tweet_id'],
                'username': reply['username'],
                'created_at': reply['created_at'],
                'text': reply['text']
            })

    return filtered_replies


filtered_replies = filter_replies_by_usernames(mentions, usernames)

for reply in filtered_replies:
    print(f"Reply ID: {reply['tweet_id']}, @{reply['username']} at {reply['created_at']}: {reply['text']}")


Reply ID: 1916996561999192560, @adekvat276 at 2025-04-28 23:22:51+00:00: @masami001001 @MINDofPepe Connect your wallet to their original website. I bought it at around 1400% at January 14. Now I have doubled the amount I bought. Because the % is changing often you get exponentially less and less reward.

But I don't think so it's worth to stake now..


In [11]:
filtered_replies

[{'tweet_id': 1916996561999192560,
  'username': 'adekvat276',
  'created_at': datetime.datetime(2025, 4, 28, 23, 22, 51, tzinfo=datetime.timezone.utc),
  'text': "@masami001001 @MINDofPepe Connect your wallet to their original website. I bought it at around 1400% at January 14. Now I have doubled the amount I bought. Because the % is changing often you get exponentially less and less reward.\n\nBut I don't think so it's worth to stake now.."}]

# Latest Top 3 Posts

In [6]:
def get_latest_top3_tweets(bearer_token, username):
    client = tweepy.Client(bearer_token=bearer_token, wait_on_rate_limit=True)

    # Get user ID from username
    try:
        user = client.get_user(username=username)
        user_id = user.data.id
    except Exception as e:
        print(f"Failed to get user ID: {e}")
        return []

    # Fetch recent tweets from the user
    try:
        response = client.get_users_tweets(
            id=user_id,
            max_results=10,  # Fetch up to 10 tweets and pick top 3
            tweet_fields=['created_at', 'public_metrics'],
            exclude=['retweets', 'replies']
        )

        tweets = response.data if response.data else []
        top3 = sorted(tweets, key=lambda x: x.created_at, reverse=True)[:3]

        result = []
        for tweet in top3:
            result.append({
                'tweet_id': tweet.id,
                'created_at': tweet.created_at,
                'text': tweet.text,
                'like_count': tweet.public_metrics['like_count'],
                'retweet_count': tweet.public_metrics['retweet_count']
            })

        return result

    except Exception as e:
        print(f"Failed to get tweets: {e}")
        return []

username = "Shift1646020"

top3_tweets = get_latest_top3_tweets(bearer_token, username)
top3_tweets

[{'tweet_id': 1918317966707528131,
  'created_at': datetime.datetime(2025, 5, 2, 14, 53, 38, tzinfo=datetime.timezone.utc),
  'text': "🚀 🚀 Crypto enthusiasts are waiting to see if Bitcoin can surpass its record high of $109,135 set earlier this year. 📈 [Nasdaq's filing aims to add popular meme coin to growing crypto ETF lineup](https://t.co/ExAHmg8782) #Bitcoin #ETFs",
  'like_count': 0,
  'retweet_count': 0},
 {'tweet_id': 1918249890444132558,
  'created_at': datetime.datetime(2025, 5, 2, 10, 23, 7, tzinfo=datetime.timezone.utc),
  'text': '🚀 Nasdaq’s filing could bring meme coin Pepe to growing crypto ETF lineup 📈📈 #PepeCoin #ETFs #CryptoMarket',
  'like_count': 0,
  'retweet_count': 0},
 {'tweet_id': 1918238631514550330,
  'created_at': datetime.datetime(2025, 5, 2, 9, 38, 23, tzinfo=datetime.timezone.utc),
  'text': '🚀$100k Bitcoin Target Reached! 🚀\nCrypto enthusiasts are excited as Bitcoin approaches its all-time high of $109,135 seen earlier this year. 📈 #Bitcoin #AltcoinETFs',


# Reply on Tweet

In [13]:
# import tweepy

# def reply_to_tweet(client, tweet_id, reply_text):
#     try:
#         response = client.create_tweet(
#             text=reply_text,
#             in_reply_to_tweet_id=tweet_id
#         )
#         print(f"Replied successfully: https://twitter.com/user/status/{response.data['id']}")
#     except tweepy.TweepyException as e:
#         print(f"An error occurred: {e}")

# reply_to_tweet(client, "1917142916343939445", "burns are where the real games happen")


import tweepy

def reply_to_tweet(client, tweet_id, reply_text):
    try:
        me = client.get_me()
        my_user_id = me.data.id

        query = f'conversation_id:{tweet_id} -is:retweet'
        existing_replies = tweepy.Paginator(
            client.search_recent_tweets,
            query=query,
            tweet_fields=['author_id'],
            max_results=100
        )

        for page in existing_replies:
            if page.data:
                for tweet in page.data:
                    if tweet.author_id == my_user_id:
                        print("Already replied to this tweet.")
                        return
        response = client.create_tweet(
            text=reply_text,
            in_reply_to_tweet_id=tweet_id
        )
        print(f"Replied successfully: https://twitter.com/user/status/{response.data['id']}")

    except tweepy.TweepyException as e:
        print(f"An error occurred: {e}")


reply_to_tweet(client, "1917142916343939445", "burns are where the real games happen")


An error occurred: 403 Forbidden
Your client app is not configured with the appropriate oauth1 app permissions for this endpoint.


# Post Tweets

In [4]:
def post_tweets(client, text, media_paths=None):
    # auth = tweepy.OAuth1UserHandler(
    #     consumer_key,
    #     consumer_secret,
    #     access_token,
    #     access_token_secret
    # )
    # api = tweepy.API(auth)

    media_ids = []
    if media_paths:
        for media_path in media_paths:
            if os.path.isfile(media_path):
                try:
                    media = api.media_upload(media_path)
                    media_ids.append(media.media_id)
                except Exception as e:
                    print(f"Error uploading {media_path}: {e}")
            else:
                print(f"File not found: {media_path}")

    try:
        if media_ids:
            response = client.create_tweet(
                text=text,
                media_ids=media_ids,
                user_auth=True
            )
        else:
            response = client.create_tweet(
                text=text,
                user_auth=True
            )
        print(f"Tweet posted successfully: https://twitter.com/user/status/{response.data['id']}")
        return response
    except Exception as e:
        print(f"An error occurred while posting the tweet: {e}")
        return None

In [5]:
text = "UK government draft rules target crypto exchanges & stablecoins 🚦\n\nUK regulators are stepping up oversight with new statutory instruments creating regulated activities like crypto asset trading exchanges and stablecoin issuance. #CryptoRegulation #UKGovTech"

In [6]:
post_tweets(client, text)

Tweet posted successfully: https://twitter.com/user/status/1920909554222588383


Response(data={'text': 'UK government draft rules target crypto exchanges &amp; stablecoins 🚦\n\nUK regulators are stepping up oversight with new statutory instruments creating regulated activities like crypto asset trading exchanges and stablecoin issuance. #CryptoRegulation #UKGovTech', 'id': '1920909554222588383', 'edit_history_tweet_ids': ['1920909554222588383']}, includes={}, errors=[], meta={})

# Replies Filter based on time

In [15]:
from datetime import datetime, timedelta, timezone

def filter_recent_replies(replies, hours=24):
    now = datetime.now(timezone.utc)
    cutoff = now - timedelta(hours=hours)

    recent_replies = sorted(
        [reply for reply in replies if reply['created_at'] >= cutoff],
        key=lambda r: r['created_at']
    )

    return recent_replies

In [16]:
filter_recent_replies(filtered_replies, hours=24)

[]

In [17]:
filtered_replies

[{'tweet_id': 1916996561999192560,
  'username': 'adekvat276',
  'created_at': datetime.datetime(2025, 4, 28, 23, 22, 51, tzinfo=datetime.timezone.utc),
  'text': "@masami001001 @MINDofPepe Connect your wallet to their original website. I bought it at around 1400% at January 14. Now I have doubled the amount I bought. Because the % is changing often you get exponentially less and less reward.\n\nBut I don't think so it's worth to stake now.."}]

# Already Commented Check

In [18]:
import tweepy
from datetime import datetime, timezone

def filter_unreplied_tweets(tweets, my_username, bearer_token):
    client = tweepy.Client(bearer_token=bearer_token, wait_on_rate_limit=True)
    unreplied = []

    for tweet in tweets:
        tweet_id = tweet['tweet_id']
        query = f'conversation_id:{tweet_id} -is:retweet'

        try:
            found_reply = False
            for response in tweepy.Paginator(
                client.search_recent_tweets,
                query=query,
                tweet_fields=['author_id', 'created_at'],
                expansions='author_id',
                user_fields=['username'],
                max_results=100
            ):
                if response.data:
                    users = {u['id']: u for u in response.includes['users']}
                    for reply in response.data:
                        author = users.get(reply.author_id)
                        if author and author.username.lower() == my_username.lower():
                            found_reply = True
                            break
                if found_reply:
                    break

            if not found_reply:
                unreplied.append(tweet)

        except Exception as e:
            print(f"Error checking tweet {tweet_id}: {e}")

    return unreplied


In [19]:
filter_unreplied_tweets(filtered_replies, "MINDofPepe", bearer_token )

[{'tweet_id': 1916996561999192560,
  'username': 'adekvat276',
  'created_at': datetime.datetime(2025, 4, 28, 23, 22, 51, tzinfo=datetime.timezone.utc),
  'text': "@masami001001 @MINDofPepe Connect your wallet to their original website. I bought it at around 1400% at January 14. Now I have doubled the amount I bought. Because the % is changing often you get exponentially less and less reward.\n\nBut I don't think so it's worth to stake now.."}]

In [1]:
import tweepy
import os
from dotenv import load_dotenv

load_dotenv()

consumer_key = os.getenv("CONSUMER_API_KEY")
consumer_secret = os.getenv("CONSUMER_API_SECRET")
access_token = os.getenv("ACCESS_TOKEN")
access_token_secret = os.getenv("ACCESS_TOKEN_SECRET")

# Set up OAuth1 authentication
auth = tweepy.OAuth1UserHandler(consumer_key, consumer_secret, access_token, access_token_secret)
api = tweepy.API(auth)

def post_tweet_with_media(text, media_paths=None):
    media_ids = []
    if media_paths:
        for media_path in media_paths:
            if os.path.isfile(media_path):
                try:
                    media = api.media_upload(media_path)
                    media_ids.append(media.media_id)
                except Exception as e:
                    print(f"Error uploading {media_path}: {e}")
            else:
                print(f"File not found: {media_path}")

    try:
        if media_ids:
            tweet = api.update_status(status=text, media_ids=media_ids)
        else:
            tweet = api.update_status(status=text)
        print(f"Tweet posted successfully: https://twitter.com/user/status/{tweet.id}")
        return tweet
    except Exception as e:
        print(f"An error occurred while posting the tweet: {e}")
        return None

# Example usage
post_tweet_with_media("Hello World!")


An error occurred while posting the tweet: 403 Forbidden
453 - You currently have access to a subset of X API V2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.x.com/en/portal/product


In [2]:
import tweepy
import os
from dotenv import load_dotenv

load_dotenv()

bearer_token = os.getenv("BEARER_TOKEN")

client = tweepy.Client(bearer_token=bearer_token)

def post_tweet(text):
    try:
        tweet = client.create_tweet(text=text, user_auth=False)
        print(f"Tweet posted successfully: https://twitter.com/user/status/{tweet.data['id']}")
        return tweet
    except Exception as e:
        print(f"An error occurred while posting the tweet: {e}")
        return None

# Example usage
post_tweet("Hello World!")


An error occurred while posting the tweet: 403 Forbidden
Authenticating with OAuth 2.0 Application-Only is forbidden for this endpoint.  Supported authentication types are [OAuth 1.0a User Context, OAuth 2.0 User Context].


In [ ]:
import tweepy
import os
from dotenv import load_dotenv
import requests
from urllib.parse import urlencode

load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
redirect_uri = os.getenv("REDIRECT_URI")

# Step 1: Generate the authorization URL
auth_url = tweepy.OAuth2UserHandler(client_id=client_id, redirect_uri=redirect_uri)
auth_url = auth_url.get_authorization_url()

print(f"Please go to this URL and authorize the application: {auth_url}")

# Step 2: Get the authorization code from the callback
auth_code = input("Enter the authorization code: ")

# Step 3: Exchange the authorization code for an access token
token_url = "https://api.twitter.com/oauth2/token"
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Authorization": f"Basic {client_id}:{client_secret}",
}
body = {
    "code": auth_code,
    "grant_type": "authorization_code",
    "redirect_uri": redirect_uri,
}
response = requests.post(token_url, headers=headers, data=urlencode(body))
response_data = response.json()
access_token = response_data["access_token"]

# Step 4: Use the access token to authenticate and post a tweet
client = tweepy.Client(bearer_token=access_token)

def post_tweet(text):
    try:
        tweet = client.create_tweet(text=text)
        print(f"Tweet posted successfully: https://twitter.com/user/status/{tweet.data['id']}")
        return tweet
    except Exception as e:
        print(f"An error occurred while posting the tweet: {e}")
        return None

# Example usage
post_tweet("Hello World!")


In [4]:
import tweepy

# Authenticate with Twitter API
client = tweepy.Client(
    consumer_key=consumer_key,
    consumer_secret=consumer_secret,
    access_token=access_token,
    access_token_secret=access_token_secret
)

# Post a tweet
try:
    response = client.create_tweet(text="Hello, world! This is a test tweet from Tweepy.")
    print("Tweet posted successfully!")
except tweepy.TweepyException as e:
    print(f"Error posting tweet: {e}")

Error posting tweet: 403 Forbidden
Your client app is not configured with the appropriate oauth1 app permissions for this endpoint.


# STATs


In [ ]:
import tweepy
from datetime import datetime

def get_my_tweets_and_replies():
    # Get the authenticated user's ID
    user_response = client.get_me()
    if user_response.data is None:
        print("Authenticated user not found.")
        return None
    user_id = user_response.data.id

    # Initialize lists to store tweets and replies
    tweets = []
    replies = []

    for response in tweepy.Paginator(
        client.get_users_tweets,
        id=user_id,
        tweet_fields=['created_at', 'in_reply_to_user_id'],
        max_results=100,
        exclude=['retweets']
    ):
        if response.data is None:
            continue
        for tweet in response.data:
            if tweet.in_reply_to_user_id is None:
                tweets.append(tweet)
            else:
                replies.append(tweet)

    last_tweet_timestamp = max((tweet.created_at for tweet in tweets), default=None)
    last_reply_timestamp = max((reply.created_at for reply in replies), default=None)

    result = {
        'total_tweets': len(tweets),
        'total_replies': len(replies),
        'last_tweet_timestamp': last_tweet_timestamp,
        'last_reply_timestamp': last_reply_timestamp,
        'tweets': [tweet.text for tweet in tweets],
        'replies': [reply.text for reply in replies]
    }

    return result


In [9]:
USERNAME = 'Shift1646020'

data = get_my_tweets_and_replies()

data

{'total_tweets': 27,
 'total_replies': 30,
 'last_tweet_timestamp': datetime.datetime(2025, 5, 6, 10, 17, 41, tzinfo=datetime.timezone.utc),
 'last_reply_timestamp': datetime.datetime(2025, 5, 5, 10, 26, 39, tzinfo=datetime.timezone.utc),
 'tweets': ['Freight train to Mar-a-Lago? Logistics firm earmarks $20m for Trump memecoin. This move comes as crypto enthusiasts wait to see if the upcoming meeting between crypto industry leaders and the US president could spark new momentum in the market.',
  'The Russian government is set to establish a crypto exchange for "super-qualified investors" under a pilot regime. This move comes as Russia looks to embrace blockchain technology, albeit cautiously. Stay tuned for further developments in digital asset regulation.',
  'In a surprising move, Russia is set to establish a crypto exchange for "super-qualified investors" under a pilot regime. This could be a significant step towards greater acceptance of digital assets in the country. Stay tuned fo

In [45]:
import requests
def last_update_api():
    url = "https://mop.rekt.life/v1/update/crypto_assets"
    try:
        response = requests.get(url)
        response.raise_for_status()

        data = response.json()
        if isinstance(data, dict) and data.get("success") and isinstance(data.get("data"), list) and len(data["data"]) == 2:
            coinmarketcap_update = data["data"][0].get("last_update", "N/A")
            solana_tracker_update = data["data"][1].get("last_update", "N/A")
            return {
                "coinmarketcap_update": coinmarketcap_update,
                "solana_tracker_update": solana_tracker_update
            }
        else:
            return {"error": "Unexpected JSON structure or missing data."}
    except requests.exceptions.RequestException as e:
        return {"error": "Request exception occurred", "details": str(e)}
    except ValueError:
        return {"error": "Failed to parse JSON response."}

In [46]:
last_update_api()

{'coinmarketcap_update': '2025-05-06T11:36:04.709444Z',
 'solana_tracker_update': '2025-05-06T11:36:03.764753Z'}

# DB

In [25]:
import psycopg2
from psycopg2 import sql

def get_rag_db_stats():
    db_params = {
        'host': '147.93.40.59',
        'port': 5432,
        'dbname': 'postgres',
        'user': 'postgres',
        'password': 'vrHwHn0VNZq5h7z2nbe4bZworOaW5GEr'
    }

    try:
        conn = psycopg2.connect(**db_params)
        cursor = conn.cursor()

        cursor.execute("""
            SELECT SUM(reltuples)::BIGINT AS total_rows
            FROM pg_class
            WHERE relkind = 'r';
        """)
        total_rows = cursor.fetchone()[0]

        cursor.execute("SELECT COUNT(*) FROM crypto_assets_embeddings;")
        total_news_items = cursor.fetchone()[0]

        cursor.execute("SELECT COUNT(*) FROM trending_tokens;")
        total_trending_tokens = cursor.fetchone()[0]

        last_news_update = cursor.fetchone()
        last_news_update = last_news_update[0] if last_news_update else None

        cursor.close()
        conn.close()

        stats = {
            'total_rows_in_rag_db': total_rows,
            'total_news_items_processed': total_news_items,
            'total_trending_tokens_processed': total_trending_tokens,
        }

        return stats

    except psycopg2.Error as e:
        print(f"Database error: {e}")
        return None


In [39]:
stats = get_rag_db_stats()

In [40]:
stats

{'total_rows_in_rag_db': 40375,
 'total_news_items_processed': 254,
 'total_trending_tokens_processed': 103}

In [42]:
def get_combined_stats_with_api():

    twitter_data = get_my_tweets_and_replies()
    db_data = get_rag_db_stats()

    api_data = last_update_api()

    if twitter_data is None or db_data is None or "error" in api_data:
        print("Error retrieving data from one or more sources.")
        return None

    combined_data = {**twitter_data, **db_data, **api_data}
    return combined_data


In [47]:
get_combined_stats_with_api()

{'total_tweets': 27,
 'total_replies': 30,
 'last_tweet_timestamp': datetime.datetime(2025, 5, 6, 10, 17, 41, tzinfo=datetime.timezone.utc),
 'last_reply_timestamp': datetime.datetime(2025, 5, 5, 10, 26, 39, tzinfo=datetime.timezone.utc),
 'tweets': ['Freight train to Mar-a-Lago? Logistics firm earmarks $20m for Trump memecoin. This move comes as crypto enthusiasts wait to see if the upcoming meeting between crypto industry leaders and the US president could spark new momentum in the market.',
  'The Russian government is set to establish a crypto exchange for "super-qualified investors" under a pilot regime. This move comes as Russia looks to embrace blockchain technology, albeit cautiously. Stay tuned for further developments in digital asset regulation.',
  'In a surprising move, Russia is set to establish a crypto exchange for "super-qualified investors" under a pilot regime. This could be a significant step towards greater acceptance of digital assets in the country. Stay tuned fo

In [48]:
def update_api():
    url = "https://mop.rekt.life/v1/update/crypto_assets"
    try:
        response = requests.post(url)
        
        if response.status_code == 200: 
            return response.json()
        else:
            return {"error": f"Request failed with status code {response.status_code}", "details": response.text}
    
    except requests.exceptions.RequestException as e:
        return {"error": "Request exception occurred", "details": str(e)}

In [50]:
update_api()

{'success': True, 'data': {'message': 'Crypto assets update started'}}

In [ ]:
import pandas as pd
from urllib.parse import quote

def add_username_to_excel(username):
    file_path = 'data/MIND.xlsx'
    
    # Read the existing Excel file
    df = pd.read_excel(file_path)
    
    # Ensure 'Profile URL' column exists
    if 'Profile URL' not in df.columns:
        raise ValueError("The Excel file must contain a 'Profile URL' column.")
    
    # Construct the profile URL
    encoded_username = quote(username)
    new_profile_url = f"https://x.com/{encoded_username}/"
    
    # Create a new DataFrame with the new profile URL
    new_row = pd.DataFrame({'Profile URL': [new_profile_url]})
    
    # Append the new row to the existing DataFrame
    updated_df = pd.concat([df, new_row], ignore_index=True)
    
    # Write the updated DataFrame back to the Excel file
    updated_df.to_excel(file_path, index=False)
    
    print(f"Username '{username}' has been added to {file_path}")

add_username_to_excel("testtestte1639")


Username 'https://x.com/testtestte1639' has been added to data/MIND.xlsx


In [29]:
import tweepy

def get_mentions_not_on_my_tweets(user_id, bearer_token, max_results=100):
    """
    Fetch tweets mentioning the user, excluding those that are replies to the user's own tweets or quote tweets of the user's own tweets.
    
    Args:
        user_id (str): Your Twitter user ID.
        bearer_token (str): Your Twitter API bearer_token.
        max_results (int): Maximum number of tweets to retrieve (5-100 per request).
    
    Returns:
        list: List of tweets mentioning the user, excluding replies to their own tweets and quote tweets of their own tweets.
    """
    # Initialize Tweepy Client
    client = tweepy.Client(bearer_token=bearer_token)
    
    # Fetch mentions with necessary fields and expansions
    mentions = client.get_users_mentions(
        id=user_id,
        tweet_fields=['id', 'text', 'author_id', 'in_reply_to_user_id', 'referenced_tweets', 'created_at'],
        expansions=['referenced_tweets.id'],
        max_results=max_results
    )
    
    # If no mentions are found, return an empty list
    if not mentions.data:
        return []
    
    # Create a dictionary of referenced tweets for easy lookup
    referenced_tweets_dict = {tweet.id: tweet for tweet in mentions.includes['tweets']} if 'tweets' in mentions.includes else {}
    
    # Filter mentions
    filtered_mentions = []
    for tweet in mentions.data:
        # Skip if it's a reply to my tweet
        if tweet.in_reply_to_user_id == user_id:
            continue
        
        # Check if it's a quote tweet of my tweet
        is_quote_of_my_tweet = False
        if tweet.referenced_tweets:
            for ref in tweet.referenced_tweets:
                if ref['type'] == 'quoted':
                    quoted_tweet_id = ref['id']
                    if quoted_tweet_id in referenced_tweets_dict:
                        quoted_tweet = referenced_tweets_dict[quoted_tweet_id]
                        if quoted_tweet.author_id == user_id:
                            is_quote_of_my_tweet = True
                            break
        
        # Include the tweet if it's neither a reply to my tweet nor a quote of my tweet
        if not is_quote_of_my_tweet:
            filtered_mentions.append({
                'id': tweet.id,
                'text': tweet.text,
                'created_at': tweet.created_at,
                'in_reply_to_user_id': tweet.in_reply_to_user_id,
                'referenced_tweets': tweet.referenced_tweets
            })
    
    return filtered_mentions

In [30]:
# Example usage
if __name__ == "__main__":

    user_id = "1917558912703684608"
    # Get mentions
    mentions = get_mentions_not_on_my_tweets(user_id, bearer_token, max_results=10)
    
    # Print results
    for mention in mentions:
        print(f"Tweet ID: {mention['id']}")
        print(f"Text: {mention['text']}")
        print(f"Created At: {mention['created_at']}")
        print(f"In Reply To User ID: {mention['in_reply_to_user_id']}")
        print("-" * 50)

Tweet ID: 1920976669365457374
Text: @Ashcryptoreal @Shift1646020 whats your thoughts on it?
Created At: 2025-05-09 22:58:22+00:00
In Reply To User ID: 3185716686
--------------------------------------------------
Tweet ID: 1920965990159897029
Text: @Bybit_Official @Shift1646020 what's your thoughts on this?
Created At: 2025-05-09 22:15:56+00:00
In Reply To User ID: 999947328621395968
--------------------------------------------------
Tweet ID: 1920961305743172085
Text: @Shift1646020 what is aisa law?
Created At: 2025-05-09 21:57:19+00:00
In Reply To User ID: 1917558912703684608
--------------------------------------------------
Tweet ID: 1920960767987257788
Text: @Shift1646020 What is ISA law?
Created At: 2025-05-09 21:55:11+00:00
In Reply To User ID: 1917558912703684608
--------------------------------------------------
Tweet ID: 1920943826878796273
Text: @Shift1646020 To stake or not to stake?
Created At: 2025-05-09 20:47:52+00:00
In Reply To User ID: 1917558912703684608
------------

In [ ]:
import requests
id = "1917558912703684608"
url = f"https://api.twitter.com/2/users/{id}/mentions"

headers = {"Authorization": f"Bearer {bearer_token}"}

response = requests.request("GET", url, headers=headers)

print(response.text)

{"data":[{"text":"@Ashcryptoreal @Shift1646020 whats your thoughts on it?","edit_history_tweet_ids":["1920976669365457374"],"id":"1920976669365457374"},{"text":"@Bybit_Official @Shift1646020 what's your thoughts on this?","edit_history_tweet_ids":["1920965990159897029"],"id":"1920965990159897029"},{"text":"@Shift1646020 what is aisa law?","edit_history_tweet_ids":["1920961305743172085"],"id":"1920961305743172085"},{"text":"@Shift1646020 What is ISA law?","edit_history_tweet_ids":["1920960767987257788"],"id":"1920960767987257788"},{"text":"@Shift1646020 To stake or not to stake?","edit_history_tweet_ids":["1920943826878796273"],"id":"1920943826878796273"},{"text":"@Shift1646020 Which one is good to purchase? eth or btc?","edit_history_tweet_ids":["1920937454506266685"],"id":"1920937454506266685"},{"text":"@Shift1646020 Which one is good to purchase?","edit_history_tweet_ids":["1920937298541089164"],"id":"1920937298541089164"},{"text":"@Shift1646020 proce of btc?","edit_history_tweet_ids

In [36]:
response.text

'{"data":[{"text":"@Ashcryptoreal @Shift1646020 whats your thoughts on it?","edit_history_tweet_ids":["1920976669365457374"],"id":"1920976669365457374"},{"text":"@Bybit_Official @Shift1646020 what\'s your thoughts on this?","edit_history_tweet_ids":["1920965990159897029"],"id":"1920965990159897029"},{"text":"@Shift1646020 what is aisa law?","edit_history_tweet_ids":["1920961305743172085"],"id":"1920961305743172085"},{"text":"@Shift1646020 What is ISA law?","edit_history_tweet_ids":["1920960767987257788"],"id":"1920960767987257788"},{"text":"@Shift1646020 To stake or not to stake?","edit_history_tweet_ids":["1920943826878796273"],"id":"1920943826878796273"},{"text":"@Shift1646020 Which one is good to purchase? eth or btc?","edit_history_tweet_ids":["1920937454506266685"],"id":"1920937454506266685"},{"text":"@Shift1646020 Which one is good to purchase?","edit_history_tweet_ids":["1920937298541089164"],"id":"1920937298541089164"},{"text":"@Shift1646020 proce of btc?","edit_history_tweet_i